In [7]:
# Usual Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

import os

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Visualization
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
#from visualkeras import layered_view

# Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
import tensorflow_hub as hub
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import layers, Sequential
from keras.callbacks import EarlyStopping

# Suppressing warnings
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
sr = 22050
general_path = '../input/gtzan-dataset-music-genre-classification/Data'
print(list(os.listdir(f'{general_path}/genres_original/')))
y, _ = librosa.load(f'{general_path}/genres_original/reggae/reggae.00036.wav', sr = sr)
        
        # 剪切沉默部分
y, _ = librosa.effects.trim(y)
print(y.shape)
print(y)

['disco', 'metal', 'reggae', 'blues', 'rock', 'classical', 'jazz', 'hiphop', 'country', 'pop']
(1440641,)
[0.02010666 0.03421702 0.04334638 ... 0.07874818 0.03808156 0.        ]


In [9]:
!python --version

Python 3.10.13


In [10]:
sr = 22050
general_path = '../input/gtzan-dataset-music-genre-classification/Data'
print(list(os.listdir(f'{general_path}/genres_original/')))

['disco', 'metal', 'reggae', 'blues', 'rock', 'classical', 'jazz', 'hiphop', 'country', 'pop']


In [11]:
# 從 TensorFlow Hub 載入 VGGish 模型
vggish = hub.load('https://www.kaggle.com/models/google/vggish/frameworks/TensorFlow2/variations/vggish/versions/1')

Attaching model 'google/vggish/tensorflow2/vggish/1' to your Kaggle notebook...


In [12]:
def mfcc_extractFeatures(audioFile):
    try:
        waveform, _ = librosa.load(audioFile, sr = sr)
        # 剪切沉默部分
        waveform, _ = librosa.effects.trim(waveform, sr = sr)
        mfccs = librosa.feature.mfcc(y=waveform)
        mfccs = sklearn.preprocessing.scale(mfccs, axis=1)
        
        if len(mfccs.shape) == 2:
            return mfccs, mfccs.shape
        else:
            print('提取的特徵不是二維的。')
            return None, None
    except Exception as e:
        # 印出更詳細的錯誤信息
        print(f"提取特徵時發生錯誤: {e} \n檔案路徑: {audioFile}")
        return None, None

In [13]:
def extractFeatures(audioFile):
    sr = 22050
    try:
        # 載入音頻文件
        waveform, _ = librosa.load(audioFile, sr=sr)
        
        # 剪切沉默部分
        waveform, _ = librosa.effects.trim(waveform)
        #print(waveform.shape)
        target_shape = 661500
        if waveform.shape[0] < target_shape:
            padding_size = target_shape - waveform.shape[0]
            # 使用擷取的片段填補到指定大小
            waveform = np.concatenate((waveform, waveform[:padding_size]))

        # 使用 VGGish 提取特徵
        print(waveform.shape)
        vggish_features = vggish(waveform).numpy()
        
        # 檢查特徵的形狀
        if len(vggish_features.shape) == 2:
            return vggish_features, vggish_features.shape
        else:
            print('提取的特徵不是二維的。')
            return None, None
    except Exception as e:
        # 印出更詳細的錯誤信息
        print(f"提取特徵時發生錯誤: {e} \n檔案路徑: {audioFile}")
        return None, None


In [14]:
# 創建一個空的列表來存儲音訊特徵和類別
data = []

# 遍歷資料夾和文件以提取特徵
for folder in os.listdir(f'{general_path}/genres_original/'):
    folderPath = os.path.join(f'{general_path}/genres_original/', folder)
    
    # 進度條顯示處理進度
    for file in tqdm(os.listdir(folderPath), desc=f'處理資料夾 {folder}', disable=True):
        filePath = os.path.join(folderPath, file)
        
        # 提取音訊特徵
        vigg, vsize = extractFeatures(filePath)
        #mfcc, msize = mfcc_extractFeatures(filePath)
        
        # 如果特徵不為空，則將特徵和類別添加到資料列表中
        if vigg is not None: #and mfcc is not None:
            data.append([vigg, folder]) 
        else:
            continue


(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1453976,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1444398,)
(1440641,)
(1445835,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1454455,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1450145,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)
(1447750,)
(1440641,)
(1453976,)
(1440641,)
(1440641,)
(1440641,)
(1440641,)

In [15]:
# Converting list into dataframe
data = pd.DataFrame(data, columns = ['vigg','Class'])
print(data.head())
# 建立一個空列表，用於存儲特徵形狀
vigg_shapes = []
#mfcc_shapes = []
# 遍歷資料列表，獲取每個特徵的形狀，並添加到列表中
for features in data['vigg']:
    vigg_shapes.append(features.shape)
# for features in data['mfcc']:
#     mfcc_shapes.append(features.shape)
# 將特徵形狀列表添加到資料框中
data['vigg_shape'] = vigg_shapes
# data['mfcc_shape'] = mfcc_shapes

# 打印資料框的前幾行
data.head()

                                                vigg  Class
0  [[-5.446558, -2.5351624, 1.9663734, -2.0244274...  disco
1  [[-4.743541, -2.3618753, 1.4738116, -1.5592064...  disco
2  [[-4.7979283, -2.5132356, 1.4603148, -1.314921...  disco
3  [[-6.3382053, -2.9159892, 2.2701082, -2.323270...  disco
4  [[-4.7428513, -2.3859158, 1.4198188, -1.495559...  disco


,vigg,Class,vigg_shape
0,"[[-5.446558, -2.5351624, 1.9663734, -2.0244274...",disco,"(93, 128)"
1,"[[-4.743541, -2.3618753, 1.4738116, -1.5592064...",disco,"(93, 128)"
2,"[[-4.7979283, -2.5132356, 1.4603148, -1.314921...",disco,"(93, 128)"
3,"[[-6.3382053, -2.9159892, 2.2701082, -2.323270...",disco,"(93, 128)"
4,"[[-4.7428513, -2.3859158, 1.4198188, -1.495559...",disco,"(93, 128)"


In [16]:
data.head(50)

,vigg,Class,vigg_shape
0,"[[-5.446558, -2.5351624, 1.9663734, -2.0244274...",disco,"(93, 128)"
1,"[[-4.743541, -2.3618753, 1.4738116, -1.5592064...",disco,"(93, 128)"
2,"[[-4.7979283, -2.5132356, 1.4603148, -1.314921...",disco,"(93, 128)"
3,"[[-6.3382053, -2.9159892, 2.2701082, -2.323270...",disco,"(93, 128)"
4,"[[-4.7428513, -2.3859158, 1.4198188, -1.495559...",disco,"(93, 128)"
5,"[[-4.5124197, -2.134753, 1.7664907, -1.2958981...",disco,"(93, 128)"
6,"[[-4.8050594, -2.185904, 2.1402478, -1.0639771...",disco,"(93, 128)"
7,"[[-5.353631, -2.2574706, 2.1121519, -1.3915612...",disco,"(93, 128)"
8,"[[-5.421294, -2.4562244, 1.6330452, -1.7536405...",disco,"(93, 128)"
9,"[[-4.5561714, -2.1428955, 1.5408643, -1.308070...",disco,"(93, 128)"


In [17]:
import pandas as pd

pickle_filename = '/kaggle/working/vggish_music_data_feature_class.pkl'
data.to_pickle(pickle_filename)

print(f'DataFrame Pickle 文件: {pickle_filename}')

data = pd.read_pickle(pickle_filename)
print(data.head())


DataFrame Pickle 文件: /kaggle/working/vggish_music_data_feature_class.pkl
                                                vigg  Class vigg_shape
0  [[-5.446558, -2.5351624, 1.9663734, -2.0244274...  disco  (93, 128)
1  [[-4.743541, -2.3618753, 1.4738116, -1.5592064...  disco  (93, 128)
2  [[-4.7979283, -2.5132356, 1.4603148, -1.314921...  disco  (93, 128)
3  [[-6.3382053, -2.9159892, 2.2701082, -2.323270...  disco  (93, 128)
4  [[-4.7428513, -2.3859158, 1.4198188, -1.495559...  disco  (93, 128)


In [18]:
import numpy as np

# Create your original array of shape (1440009,)
original_array = np.random.rand(5,1)  # This is just an example, you would use your actual data here

# Define the number of elements to pad
num_elements_to_pad = 10 - 5

# Calculate the number of segments to repeat
num_segments = num_elements_to_pad // len(original_array)

# Calculate the remaining elements after repetition
remaining_elements = num_elements_to_pad % len(original_array)

# Repeat the original array segments
padded_array = np.concatenate((original_array, original_array[:num_elements_to_pad]))

print(padded_array)  # Output: (1440641,)


[[0.58502197]
 [0.05960497]
 [0.77990265]
 [0.63324908]
 [0.15790213]
 [0.58502197]
 [0.05960497]
 [0.77990265]
 [0.63324908]
 [0.15790213]]
